# Python Sorted Collections

Grant Jenks

http://www.grantjenks.com/docs/sortedcontainers/introduction.html

* heapq
* bisect
* queue

collections.OrderedDict is *not* what you think it is

none of these fit the bill

Python has to depend on external (non stdlib) 

# Proposed new types

* SortedList
* SortedDict
* SortedSet


# History


* Blist - 2006 - B-Tree based replacement for built in `list`, pretty good  API
* sortedcollection - 2010 - by Raymond Hettinger: linked in the stdlib docs (slows down after 10k items)
  * Mostly meant for read-only workloads
* bintrees = 2010 - Multiple tree implementations, extends blist
* banyan - 2013 - short-lived but very fast (C++)
* skiplistcollections - 2013 - pure python, and very fast. Not a tree

# PYPI yields way too many answers to this question
and google `site:` is better

> There should be one right way to do it

# Enter SortedContainers
> insert joke about why we need another standard
 
 pure python, but "as fast as C extensions"
 tested (100% coverage!)
 
> the slides have some graphs comparing the performance of SortedContainers against other implementations

# Features

* pop
* bisect_right
* irange - yeilds all keys in sorted order in that range
* iloc - the same as pandas
* islice - positional index slicing, yeilds iterator over a range

# Under the hood
Why is it so fast?


`bisect` does the heavy lifting

## Basic structure
* `_lists`: a list of sub-lists, maintained in sorted order (a B-tree)
* `_maxes`: a list of the maximum values in each sub-list
* `load_factor`: 
  * when a sub-list grows too large, the list is split in half
  * when a sub-list shrinks too small, it is combined with the adjacent lists


# Builtin types are __fast__
because they're C, and they've had years of optimization

* Program your interpretter


# Memory is tiered
* ~12 registerd
* L1 cache: ~32KB
* L2 cache: 256KB
* L3 cache, shared 8mb (if you even have it)
* many gigabytes of RAM

* The higher you get in memory hierarchy, the slower it is
* Because of how CPUs work, sequential memory access is very fast (they predict what you're doing)
* Random, or data-dependent (pointers) lookups are slow because they don't have this advantage

At large scales, binary trees do more data-dependent memory lookups than SortedList does




> Look up Grant Jenks' blog post about this

__Note from Raphael__: I had a long hallway conversation with Raymond Hettinger (Python core dev... author of `Collections` module) about this talk, and I extracted some points that should be added here:

* There is no singular sorted collection implementation that is optimal for all use cases
  * e.g. Mostly reading vs Mostly writing
* Some methods exposed in the `SortedContainer` package (like `bisect`) are just forwarding along implementation details, and probably don't belong in the public API.
  * the usage of these should probably be avoided in most cases
  * The reasoning here, is that by exposing these things you encourage a pattern of usage that strays from the purpose of the package/module/class/etc
* There's some violation here of the original concepts behind the original types that have "sorted" counterparts here
  * For instance, a `set` doesn't really have any business being sorted
    * `sets` are designed for removing duplicates, and quick membership tests
* the impressive performance, and testimonials presented about `SortedContainers` will probably lead to a call to add this to the stdlib. There's some interesting reasons why this isn't straightforward:
  * Python doesn't want to be opinionated, especially when there are different optimal ways do to things depending on the use case
    * the preferred route is to provide powerful building blocks to make these things possible
      * e.g.: requests is better off outside of stdlib
  * The API this package exposes (in its current form) promotes some less than ideal usages
    * generally, this leads to feature requests, bugs, etc that seek to optimize these cases, while degrading the actual intended case
    * For example: if you added a numeric index to *all* dictionaries, people who aren't even using that functionality have to deal with the added memory consumption for no added benefit
  * This is probably going to launch a big "war" about adding `SortedCollections` to the stdlib, and it will be interesting to watch the drama unfold
    * grab your popcorn
    